In [ ]:
# #auto-completion
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install - user
# from jedi import settings
# settings.case_insensitive_completion = True

In [ ]:
# #Install the kubeflow components.
# USER_FLAG = "--user"
# # Install ai platform and kfp
# !pip3 install {USER_FLAG} google-cloud-aiplatform==1.3.0 --upgrade
# !pip3 install {USER_FLAG} kfp --upgrade
# !pip install google_cloud_pipeline_components

In [ ]:
# ### Enable the APIs if they are not enabled
# !gcloud services enable compute.googleapis.com         \
#                        containerregistry.googleapis.com  \
#                        aiplatform.googleapis.com  \
#                        cloudbuild.googleapis.com \
#                        cloudfunctions.googleapis.com

In [ ]:
# ###Import the libraries
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)

from kfp.v2 import compiler
from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [ ]:
# ###Set up the global variables

# PATH=%env PATH
# %env PATH={PATH}:/home/jupyter/.local/bin
# REGION="europe-west1"

# # Get projet name
# shell_output=!gcloud config get-value project 2> /dev/null
# PROJECT_ID=shell_output[0]

# # Set bucket name
# BUCKET_NAME="gs://"+PROJECT_ID+"-bucket-smsquality

# # Create bucket
# PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root_wine/"
# PIPELINE_ROOT

# USER_FLAG = "--user"
# #!gcloud auth login if needed

In [ ]:
# ### Install a virtual env and use it inside the JupyterLab
# #Install virtual env 
# python -m pip install --user virtualenv
# echo "create env"
# python -m venv vertex_venv
# #Add kernel to jupyter
# echo "Add kernel to jupyter"
# ipython kernel install --name "vertex_env" --user

In [ ]:
##Load the data

@component(
    packages_to_install=["numpy","pandas", "sklearn", "tensorflow", "matplotlib", "seaborn"],
    base_image="python:3.9",
    output_component_file="get_data.yaml"
)

def get_data(
    url: str,
    dataset_train: Output[Dataset],
    dataset_test: Output[Dataset]
):
    import pandas as pd
    import numpy as np
    import numpy as np
    import pandas as pd # pylint: disable=unused-import
    from sklearn.model_selection import train_test_split as tts
    import tensorflow as tf
    import tensorflow_hub as hub # pylint: disable=unused-import
    import tensorflow_text as text # pylint: disable=unused-import
    from sklearn.metrics import classification_report  # pylint: disable=ungrouped-imports
    from sklearn.metrics import confusion_matrix
    from matplotlib import pyplot as plt
    import seaborn as sn

    df_spam = pd.read_csv(url)
    
    # Create the label collumn
    df_spam['target']=df_spam['Class'].apply(lambda x: 1 if x=='spam' else 0)
    df_spam = df_spam.drop(['Class'], axis=1)
  
    train, test = tts(df_spam, test_size=0.3)
    train.to_csv(dataset_train.path + ".csv" , index=False, encoding='utf-8-sig')
    test.to_csv(dataset_test.path + ".csv" , index=False, encoding='utf-8-sig')

In [ ]:
##### Train the model

@component(
    packages_to_install = ["numpy","pandas", "sklearn", "tensorflow", "matplotlib", "seaborn"], 
    base_image="python:3.9",
)
def train_sms(dataset: Input[Dataset], model: Output[Model]):

    import pandas as pd
    import pickle
    import numpy as np
    import pandas as pd # pylint: disable=unused-import
    from sklearn.model_selection import train_test_split
    import tensorflow as tf
    import tensorflow_hub as hub # pylint: disable=unused-import
    import tensorflow_text as text # pylint: disable=unused-import
    from sklearn.metrics import classification_report  # pylint: disable=ungrouped-imports
    from sklearn.metrics import confusion_matrix
    from matplotlib import pyplot as plt
    import seaborn as sn
    bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
    bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

    data = pd.read_csv(dataset.path+".csv")
    ####

    # Bert layers
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Neural network layers
    lay = tf.keras.layers.Dense(64, activation='relu', name="dense1")(outputs['pooled_output'])
    lay = tf.keras.layers.Dropout(0.2, name="dropout1")(lay)
    lay = tf.keras.layers.Dense(32, activation='relu', name="dense2")(lay)
    lay = tf.keras.layers.Dropout(0.2, name="dropout")(lay)
    lay = tf.keras.layers.Dense(1, activation='sigmoid', name="output2")(lay)

    # Use inputs and outputs to construct a final model
    model = tf.keras.Model(inputs=[text_input], outputs=[lay])

    # print summary
    model.summary()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # train the model
    model_rf = model.fit(data['sms'], data['target'], epochs=2 )

    # #####
    # model_rf = RandomForestClassifier(n_estimators=10)
    # model_rf.fit(
    #     data.drop(columns=["target"]),
    #     data.target,
    # )

    model.metadata["framework"] = "RF"
    file_name = model.path + f".pkl"
    with open(file_name, 'wb') as file:  
        pickle.dump(model_rf, file)

In [ ]:
#### Evaluate the model

@component(
    packages_to_install = ["numpy", "pandas", "sklearn", "tensorflow", "matplotlib", "seaborn"], base_image="python:3.9")
def smsquality_evaluation(
    test_set:  Input[Dataset],
    rf_smsquality_model: Input[Model],
    thresholds_dict_str: str,
    metrics: Output[ClassificationMetrics],
    kpi: Output[Metrics]
) -> NamedTuple("output", [("deploy", str)]):

    import pandas as pd
    import logging 
    import pickle
    from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, classification_report
    import json
    import typing
    import numpy as np    
    from sklearn.model_selection import train_test_split
    import tensorflow as tf
    import tensorflow_hub as hub
    import tensorflow_text as text    
    from matplotlib import pyplot as plt
    import seaborn as sn
    from sklearn.ensemble import RandomForestClassifier
       
    def threshold_check(val1, val2):
        cond = "false"
        if val1 >= val2 :
            cond = "true"
        return cond

    data = pd.read_csv(test_set.path+".csv")
    ####

    file_name = rf_smsquality_model.path + ".pkl"
    with open(file_name, 'rb') as file:  
        model = pickle.load(file)
    
    y_test = data.drop(columns=["target"])
    y_target=data.target
    y_pred = model.predict(y_test)
    
    y_scores =  model.predict_proba(data.drop(columns=["target"]))[:, 1]
    fpr, tpr, thresholds = roc_curve(
         y_true=data.target.to_numpy(), y_score=y_scores, pos_label=True
    )
    metrics.log_roc_curve(fpr.tolist(), tpr.tolist(), thresholds.tolist())  
    
    metrics.log_confusion_matrix(
       ["False", "True"],
       confusion_matrix(
           data.target, y_pred
       ).tolist(), 
    )
    
    accuracy = accuracy_score(data.target, y_pred.round())
    thresholds_dict = json.loads(thresholds_dict_str)
    rf_smsquality_model.metadata["accuracy"] = float(accuracy)
    kpi.log_metric("accuracy", float(accuracy))
    deploy = threshold_check(float(accuracy), int(thresholds_dict['roc']))

    deploy = model
    return deploy

In [ ]:
### Deploy the model

@component(
    packages_to_install=["google-cloud-aiplatform", "scikit-learn==1.0.0",  "kfp"],
    base_image="python:3.9",
    output_component_file="model_smsquality_coponent.yml"
)
def deploy_smsequality(
    model: Input[Model],
    project: str,
    region: str,
    serving_container_image_uri : str, 
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model]
):
    from google.cloud import aiplatform
    aiplatform.init(project=project, location=region)

    DISPLAY_NAME  = "smsquality"
    MODEL_NAME = "smsquality-rf"
    ENDPOINT_NAME = "smsquality_endpoint"
    
    def create_endpoint():
        endpoints = aiplatform.Endpoint.list(
        filter='display_name="{}"'.format(ENDPOINT_NAME),
        order_by='create_time desc',
        project=project, 
        location=region,
        )
        if len(endpoints) > 0:
            endpoint = endpoints[0]  # most recently created
        else:
            endpoint = aiplatform.Endpoint.create(
            display_name=ENDPOINT_NAME, project=project, location=region
        )
    endpoint = create_endpoint()   
    
    
    #Import a model programmatically
    model_upload = aiplatform.Model.upload(
        display_name = DISPLAY_NAME, 
        artifact_uri = model.uri.replace("model", ""),
        serving_container_image_uri =  serving_container_image_uri,
        serving_container_health_route=f"/v1/models/{MODEL_NAME}",
        serving_container_predict_route=f"/v1/models/{MODEL_NAME}:predict",
        serving_container_environment_variables={
        "MODEL_NAME": MODEL_NAME,
    },       
    )
    model_deploy = model_upload.deploy(
        machine_type="n1-standard-4", 
        endpoint=endpoint,
        traffic_split={"0": 100},
        deployed_model_display_name=DISPLAY_NAME,
    )

    # Save data to the output params
    vertex_model.uri = model_deploy.resource_name

In [ ]:
####### Create the pipeline

from datetime import datetime
TIMESTAMP =datetime.now().strftime("%Y%m%d%H%M%S")
DISPLAY_NAME = 'pipeline-smsquality-job{}'.format(TIMESTAMP)


####
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root= PIPELINE_ROOT,
    # A name for the pipeline. Use to determine the pipeline Context.
    name= pipeline-smsquality
    
)
def pipeline(
    url: str =  "SMSCollection.csv"  #storage path 
    project: str = PROJECT_ID,
    region: str = REGION, 
    display_name: str = DISPLAY_NAME,
    api_endpoint: str = REGION+"-aiplatform.googleapis.com",
    thresholds_dict_str: str = '{"roc":0.8}',
    serving_container_image_uri: str = "europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
    ):
    
    data_op = get_data(url)
    train_model_op = train_sms(data_op.outputs["dataset_train"])
    model_evaluation_op = smsquality_evaluation(
        test_set=data_op.outputs["dataset_test"],
        rf_smsquality_model=train_model_op.outputs["model"],
        thresholds_dict_str = thresholds_dict_str, # I deploy the model anly if the model performance is above the threshold
    )
    
    with dsl.Condition(
        model_evaluation_op.outputs["deploy"]=="true",
        name="deploy-smsquality",
    ):
           
        deploy_model_op = deploy_smsequality(
        model=train_model_op.outputs['model'],
        project=project,
        region=region, 
        serving_container_image_uri = serving_container_image_uri,
        )

SyntaxError: ignored

In [ ]:
#### Run the pipeline
compiler.Compiler().compile(pipeline_func=pipeline, package_path='ml_smsquality.json')



In [ ]:
#### Create a run using the job spec file previously generated
start_pipeline = pipeline_jobs.PipelineJob(
    display_name="smsquality-pipeline",
    template_path="ml_smsquality.json",
    enable_caching=True,
    location=REGION,
)



In [ ]:
#### Run the pipeline

start_pipeline.run()